<a href="https://colab.research.google.com/github/nalivaikaanastasiya-dev/HTTP-Web-Search-Briefing-Bot/blob/main/client_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
import requests
from datetime import date

BASE = "http://127.0.0.1:8000"
TOKEN = "Bearer devtoken123"

def post(path, payload):
    r = requests.post(
        BASE + path,
        json=payload,
        headers={"Authorization": TOKEN},
        timeout=30,
    )
    r.raise_for_status()
    return r.json()

topic = " ".join(sys.argv[1:]) or "AI agents in 2025"

results = post("/tools/search_web", {"query": topic, "k": 5})
docs = [post("/tools/fetch_readable", {"url": r["url"]}) for r in results[:3]]

summary = post(
    "/tools/summarize_with_citations",
    {"topic": topic, "docs": docs},
)

md = "# Brief\n\n"
for b in summary["bullets"]:
    md += f"- {b}\n"

md += "\n## Sources\n"
for s in summary["sources"]:
    md += f"[{s['i']}] {s['title']} — {s['url']}\n"

fname = f"brief_{date.today()}.md"
out = post("/tools/save_markdown", {"filename": fname, "content": md})

print("Saved to:", out["path"])
